In [8]:
# Calibration of CIR85 model to Euribor Market Rates

#Import necessary packages
import math
import numpy as np
np.set_printoptions(suppress=True,
                    formatter={'all': lambda x: '%7.6f' % x})
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['font.family'] = 'serif'
import scipy.interpolate as sci
from scipy.optimize import fmin

#Import dependencies
from ipynb.fs.full.CIR_zcb_valuation import B

#Market data for calibration
#Updated with Eonia rate and Euribor rates on 3 December 2020. Should probably be Swedish rates since ABB stock
t_list = np.array((1, 7, 30, 90, 180, 360)) / 360.
r_list =  np.array((-0.471, -0.548, -0.539, -0.527, -0.510, -0.488)) / 100

#Calculate zero rates
factors = (1 + t_list * r_list)
zero_rates = 1 / t_list * np.log(factors)

#Today's rate = 1-day short rate
r0 = r_list[0] # set to Eonia rate 

#Interpolation of Market Data              # Find the B-spline representation of a 1-D curve.
tck = sci.splrep(t_list, zero_rates, k=3)  # Cubic splines to interpolate. k = degree of spline fit.
tn_list = np.linspace(0.0, 1.0, 24)        
ts_list = sci.splev(tn_list, tck, der=0)   #Evaluate a B-spline or its derivatives
de_list = sci.splev(tn_list, tck, der=1)

#Forward rate transformation
f = ts_list + de_list * tn_list

# CIR model calulation of Forward Rates
def CIR_forward_rate(opt):
    ''' Function for forward rates in CIR85 model.

    Parameters
    ==========
    kappa_r: float
        mean-reversion factor
    theta_r: float 
        long-run mean
    sigma_r: float
        volatility factor

    Returns
    =======
    forward_rate: float
        forward rate
    '''
    kappa_r, theta_r, sigma_r = opt
    t = tn_list
    g = np.sqrt(kappa_r ** 2 + 2 * sigma_r ** 2)
    sum1 = ((kappa_r * theta_r * (np.exp(g * t) - 1)) /
          (2 * g + (kappa_r + g) * (np.exp(g * t) - 1)))
    sum2 = r0 * ((4 * g ** 2 * np.exp(g * t)) /
            (2 * g + (kappa_r + g) * (np.exp(g * t) - 1)) ** 2)
    forward_rate = sum1 + sum2
    return forward_rate

#Loss Function or Error Function if you prefer
def CIR_error_function(opt):
    ''' Error function for CIR85 model calibration. '''
    kappa_r, theta_r, sigma_r = opt
    if 2 * kappa_r * theta_r < sigma_r ** 2:
        return 100
    if kappa_r < 0 or theta_r < 0 or sigma_r < 0.001:
        return 100
    forward_rates = CIR_forward_rate(opt)
    MSE = np.sum((f - forward_rates) ** 2) / len(f)
    # print opt, MSE
    return MSE


#Calibration Procedure
def CIR_calibration():
    opt = fmin(CIR_error_function, [1.0, 0.02, 0.1],
            xtol=0.00001, ftol=0.00001,
            maxiter=300, maxfun=500)
    return opt